In [3]:
import sys
sys.path.append('..')

from src.datasets import I2PLoader, MMALoader, TocixChatLoader

In [16]:
i2p = I2PLoader(path='/home/artanisax/Github/Artanisax/concept_gen_eva/datasets/i2p/i2p_benchmark.csv')
mma = MMALoader(path='/home/artanisax/Github/Artanisax/concept_gen_eva/datasets/MMA/mma-diffusion-nsfw-adv-prompts.csv')
toxic_chat = TocixChatLoader(path='/home/artanisax/Github/Artanisax/concept_gen_eva/datasets/toxic_chat/toxic-chat_annotation_all_0124.csv')

In [17]:
prompts = toxic_chat.extract_prompt()
with open('toxic_chat.txt', 'w') as file:
    for prompt in prompts:
        file.write(prompt.replace('\n', ' ')+'\n')

In [5]:
data = ''
with open('requirements.txt', 'r') as file:
    for line in file:
        pos = line.find('==')
        if pos != -1:
            data += line[:pos] + '\n'
        else:
            data += line + '\n'
print(data)
with open('requirements.txt', 'w') as file:
    file.write(data)

absl-py
accelerate
asttokens
certifi
charset-normalizer
clip
comm
contourpy
cycler
debugpy
decorator
diffusers
embedding-reader
exceptiongroup
executing
filelock
fire
fonttools
fsspec
ftfy
grpcio
huggingface-hub
idna
importlib_metadata
importlib_resources
ipykernel
ipython
jedi
Jinja2
jupyter_client
jupyter_core
kiwisolver
lightning-utilities
Markdown
MarkupSafe
matplotlib
matplotlib-inline
mpmath
multidict
nest_asyncio
networkx
numpy
nvidia-cublas-cu12
nvidia-cuda-cupti-cu12
nvidia-cuda-nvrtc-cu12
nvidia-cuda-runtime-cu12
nvidia-cudnn-cu12
nvidia-cufft-cu12
nvidia-curand-cu12
nvidia-cusolver-cu12
nvidia-cusparse-cu12
nvidia-nccl-cu12
nvidia-nvjitlink-cu12
nvidia-nvtx-cu12
packaging
pandas
parso
pexpect
pickleshare
pillow
pip
platformdirs
prompt-toolkit
protobuf
psutil
ptyprocess
pure-eval
pyarrow
Pygments
pyparsing
python-dateutil
pytz
PyYAML
pyzmq
regex
requests
safetensors
seaborn
setuptools
six
stack-data
sympy
tensorboard
tensorboard-data-server
termcolor
tokenizers
torch
torchaud